# 前言

- 下面是一个利用梯度直接实现的LeNet-5的手写数字识别程序，其中的核心部分，在本文采用封装的方式实现。实际上Torch的不仅仅封装，还做了很多优化的操作，这里列出代码，仅仅是理解与说明Module与Layer的封装原理；以及每个封装的模块封装的主要功能与操作。
- 为了计算时间，只训练了100轮，每轮的训练样本分成6个批次，每个批次10000个样本。

In [1]:
import struct
import numpy as np
import torch
import math

# 读取图片
def load_image_fromfile(filename):
    with open(filename, 'br') as fd:
        # 读取图像的信息
        header_buf = fd.read(16)   # 16字节，4个int整数
        # 按照字节解析头信息（具体参考python SL的struct帮助）
        magic_, nums_, width_, height_ = struct.unpack('>iiii', header_buf)  # 解析成四个整数：>表示大端字节序，i表示4字节整数
        # 保存成ndarray对象
        imgs_ = np.fromfile(fd, dtype=np.uint8)
        imgs_ = imgs_.reshape(nums_, height_, width_)
    return imgs_

# 读取标签
def load_label_fromfile(filename):
    with open(filename, 'br') as fd:
        header_buf = fd.read(8) 
        magic, nums = struct.unpack('>ii' ,header_buf) 
        labels_ = np.fromfile(fd, np.uint8) 
    return labels_

# 读取训练集
train_x = load_image_fromfile("datasets/train-images.idx3-ubyte")
train_y = load_label_fromfile("datasets/train-labels.idx1-ubyte")
train_x = train_x.astype(np.float64)
train_y = train_y.astype(np.int64)
# 读取测试集
test_x = load_image_fromfile("datasets/t10k-images.idx3-ubyte")
test_y = load_label_fromfile("datasets/t10k-labels.idx1-ubyte")

# 1. 定义训练参数（3层卷积，2层全链接）
#   1.1. 1 @28 * 28 ->   6@28 * 28:  6@5*5 的卷积核  -> 6 @ 14 * 14(池化)
#   1.2. 6@14 * 14 ->   16@10 * 10:  16@5*5 的卷积核  -> 10 @ 5 * 5(池化)
#   1.3. 16@5 * 5 ->   120@1 * 1:  120@5*5 的卷积核  （没有池化）
#   1.4. 120 * 84  
#   1.5. 84 * 10   (输出10个特征，分类0-9十个数字)
# 1.1
w_6_5_5 = torch.Tensor(6, 1, 5, 5)   #（C_out, C_in, H_k, W_k）
b_6_5_5 = torch.Tensor(6)               # (C_out)   # 卷积核也可以不使用偏置项的
# 初始化（模仿Torch的源代码，自己采用正态分布，每次计算都是无穷大）
stdv = 1.0 / math.sqrt(1 * 5 * 5)
w_6_5_5.data.uniform_(-stdv, stdv)
b_6_5_5.data.uniform_(-stdv, stdv)
# print(w_6_5_5)

# 1.2 
w_16_5_5 = torch.Tensor(16, 6, 5, 5)
b_16_5_5 = torch.Tensor(16)
# 初始化
stdv = 1.0 / math.sqrt(6 * 5 * 5)
w_16_5_5.data.uniform_(-stdv, stdv)
b_16_5_5.data.uniform_(-stdv, stdv)

# 1.3 
w_120_5_5 = torch.Tensor(120, 16, 5, 5)
b_120_5_5 = torch.Tensor(120) 
# 初始化
stdv = 1.0 / math.sqrt(16 * 5 * 5)
w_120_5_5.data.uniform_(-stdv, stdv)
b_120_5_5.data.uniform_(-stdv, stdv)

# 1.4
w_120_84 = torch.Tensor(84, 120) 
b_120_84 = torch.Tensor(84) 
# 初始化
stdv = 1.0 / math.sqrt(120)   # 使用输入的特征数作为均匀分布的计算基数
w_120_84.data.uniform_(-stdv, stdv)
b_120_84.data.uniform_(-stdv, stdv)

# 1.5
w_84_10 =torch.Tensor(10, 84) 
b_84_10 = torch.Tensor(10)
# 初始化
stdv = 1.0 / math.sqrt(84)
w_84_10.data.uniform_(-stdv, stdv)
b_84_10.data.uniform_(-stdv, stdv)


# print(w_16_5_5)
w_6_5_5.requires_grad = True
b_6_5_5.requires_grad = True
# 1.2 
w_16_5_5.requires_grad = True
b_16_5_5.requires_grad = True
# 1.3 
w_120_5_5.requires_grad = True
b_120_5_5.requires_grad = True
# 1.4
w_120_84.requires_grad = True
b_120_84.requires_grad = True
# 1.5
w_84_10.requires_grad = True
b_84_10.requires_grad = True

# 2. 定义forward模型(为了反复调用，封装成函数)
@torch.enable_grad()
def lenet5_forward(input):
    """
    input的格式：4-D（N, 1, 28, 28）：N表示每批次的样本数量
    out的格式：与input相同4-D（N, 10）：N表示每批次的样本数量
    """
    # 1.1 
    o_c1 = torch.nn.functional.conv2d(input=input, weight=w_6_5_5, bias=b_6_5_5, padding = 2)  # 原始图像28*28
    o_a1 = torch.nn.functional.relu(o_c1)
    o_p1 = torch.nn.functional.max_pool2d(input= o_a1, kernel_size=(2,2))
    o1 = o_p1
    # 1.2
    o_c2 = torch.nn.functional.conv2d(input=o1, weight=w_16_5_5, bias=b_16_5_5)
    o_a2 = torch.nn.functional.relu(o_c2)
    o_p2 = torch.nn.functional.max_pool2d(input= o_a2, kernel_size=(2,2))
    o2 = o_p2
    # 1.3
    o_c3 = torch.nn.functional.conv2d(input=o2, weight=w_120_5_5, bias=b_120_5_5)
    o_a3 = torch.nn.functional.relu(o_c3)
    # 无池化
    # o3 = o_a3.squeeze()    # 格式转换(把最后的1*1直接降维掉），转换为60000 * 120
    o3 = o_a3.view(o_a3.shape[0], o_a3.shape[1])
    # 1.4
    o_c4 = torch.nn.functional.linear(o3, w_120_84, b_120_84)
    o_a4 = torch.nn.functional.relu(o_c4)
    o4 = o_a4
    # 1.5
    o_c5 = torch.nn.functional.linear(o4, w_84_10, b_84_10)
    o_a5 = torch.log_softmax(o_c5, dim=1)
    o5 = o_a5
    return  o5 

# 3. 定义损失模型（封装成函数）
@torch.enable_grad()
def loss_model(out, target):
    loss_ = torch.nn.functional.cross_entropy(out, target)
    return loss_


# 为了速度取1000个样本训练
# train_x = train_x[0:10]
# train_y = train_y[0:10]
# 训练集
x = torch.Tensor(train_x).view(train_x.shape[0], 1, train_x.shape[1], train_x.shape[2])   # N,C,W,H
y = torch.LongTensor(train_y)
# # 测试集
t_x =  torch.Tensor(test_x).view(test_x.shape[0], 1, test_x.shape[1], test_x.shape[2])   # N,C,W,H
t_y =  torch.LongTensor(test_y)

# 训练超参数
# 学习率
learn_rate = 0.001
# 训练轮数
epoch = 100
# 没批样本数
batch_size = 10000
# 批次计算
batch_num = len(train_y) // batch_size

# 轮次循环
for e in range(epoch):
    # 批次循环
    for idx in range(batch_num):
        # 批次样本
        start = idx *batch_size
        end = (idx + 1) * batch_size
        b_x = x[start: end]
        b_y = y[start: end]
        # 计算输出
        b_y_ = lenet5_forward(b_x)
#         break
        # 计算损失
        l_ = loss_model(b_y_, b_y)
        # 计算梯度
        l_.backward(retain_graph=True)
#         print(w_6_5_5.grad)
        # 梯度更新(使用上下文管理器，进制对运算实现图跟踪)
        with torch.autograd.no_grad():
            w_6_5_5 -= learn_rate * w_6_5_5.grad
            b_6_5_5 -= learn_rate * b_6_5_5.grad

            w_16_5_5 -= learn_rate * w_16_5_5.grad
            b_16_5_5 -= learn_rate * b_16_5_5.grad

            w_120_5_5 -= learn_rate * w_120_5_5.grad
            b_120_5_5 -= learn_rate * b_120_5_5.grad

            w_120_84 -= learn_rate * w_120_84.grad
            b_120_84 -= learn_rate * b_120_84.grad
            
            w_84_10 -= learn_rate * w_84_10.grad
            b_84_10 -= learn_rate * b_84_10.grad
            
            # 复原梯度
            w_6_5_5.grad.zero_()
            b_6_5_5.grad.zero_()
            
            w_16_5_5.grad.zero_()
            b_16_5_5.grad.zero_()
            
            w_120_5_5.grad.zero_()
            b_120_5_5.grad.zero_()
            
            w_120_84.grad.zero_()
            b_120_84.grad.zero_()
            
            w_84_10.grad.zero_()
            b_84_10.grad.zero_()
            
    # 每一轮次完毕，输出损失度与测试集准确率
    if e % 10 ==0:
        print(F"第{e:03d}轮")
        print(F"\t损失值：{l_:8.6f}",end="")  
        # 测试集测试
        with torch.autograd.no_grad():   
            predict = lenet5_forward(t_x)
            # 计算准确率
            y_ = predict.argmax(dim=1)
            correct_rate = (y_ == t_y).float().mean()
            print(F"\t测试集准确率：{correct_rate*100: 6.2f}%")
print("------训练完毕------") 


第000轮
	损失值：2.358580	测试集准确率： 15.48%
第010轮
	损失值：0.888810	测试集准确率： 73.71%
第020轮
	损失值：0.526765	测试集准确率： 83.67%
第030轮
	损失值：0.414707	测试集准确率： 87.15%
第040轮
	损失值：0.345243	测试集准确率： 89.41%
第050轮
	损失值：0.307811	测试集准确率： 90.55%
第060轮
	损失值：0.277965	测试集准确率： 91.49%
第070轮
	损失值：0.254911	测试集准确率： 92.05%
第080轮
	损失值：0.236175	测试集准确率： 92.63%
第090轮
	损失值：0.220653	测试集准确率： 93.08%
------训练完毕------


- 上面训练过程的核心是：
    1. 决策模型输出：
        - `b_y_ = lenet5_forward(b_x)`
    2. 损失计算：
        - `l_ = loss_model(b_y_, b_y)`
    3. 梯度计算：
        - `l_.backward(retain_graph=True)`
    4. 梯度更新：
        - `w_6_5_5 -= learn_rate * w_6_5_5.grad`

# 容器与Module封装

- 对其中决策运算的封装就是Module类，这个类主要提供的是一种封装框架，具体的决策过程实现需要根据需求个性化来实现，这个封装的好处就是利用Python的可调用对象实现决策计算输出，并在调用的时候隐藏繁琐的实现细节，实现简洁的训练过程调用（与我们封装成函数的目的一样，只是封装成函数比较low一点）。
    - m = torch.nn.Module()
    - result = m(input)     # 预测结果，或者输入样本的计算结果

## Module类的基本使用模式

- Module的使用模式由其设计决定，一般使用模式两个要素：
    - 通过继承Module，重载forward函数，实现决策输出运算；
        - 在构造器`__init__`函数中提供运算需要的初始化对象；
        - forward的参数根据使用自己确定；
    - 调用Module完成决策模型运算。

- 下面是Module的使用例子代码   

In [3]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 初始化
        
    def forward(self):
        return "决策输出计算结果"


m = PerceptronModule()

result = m()
print(result)

决策输出计算结果


## 运算过程及其管理

- Module对运算过程的管理包含：
    1. 训练参数的定义
    2. 训练参数的属性设置
    3. 训练参数的初始化
    4. 决策计算过程
    5. 决策过程求导的处理（中间变量的导数）
    
- 下面先看看运算过程，例子代码是感知器的实现。
    - 其中所有的管理采用过程式管理，所有管理都是单独管理（需要直接访问变量来管理）

In [19]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 需要训练的参数定义
        self.w = torch.Tensor(1, 4) 
        self.b = torch.Tensor(1) 
        
        # 2. 需要训练的参数设置
        self.w.requires_grad = True
        self.b.requires_grad = True
        
        # 3. 需要训练的参数初始化
        stdv = 1.0 / math.sqrt(4)   
        self.w.data.uniform_(-stdv, stdv)
        self.b.data.uniform_(-stdv, stdv)

        
    def forward(self, x):
        # 4. 计算过程的实现
        out = torch.nn.functional.linear(x, self.w, self.b)
        y_ = torch.sigmoid(out)
        return y_


import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

# Module对象
m = PerceptronModule()
# Module对象调用
y_ = m(x)

# 使用Module计算损失函数
loss = torch.nn.functional.mse_loss(y_, y)

# 优化损失-求导
loss.backward()

print(m.parameters())
print(list(m.parameters()))   # 采用非模块方式，对参数，求导等管理比较麻烦

<generator object Module.parameters at 0x139692eb8>
[]


## 子模块管理

- 为了便于管理每个运算过程，采用子模块管理每个运算过程。而不是像上面采用的非模块化封装。
    - 把一个运算过程设计成一个独立的对象
        - Function：无训练参数
        - Layer：有训练参数（通过访问Layer对象的属性管理）
- 下面是采用子模块管理的例子代码
    - 采用子模块管理的好处就是通过模块名，可以管理相关的所有参数、数据与数据计算过程；

In [21]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 子模块对象
        # （简单的定义隐藏了参数定义，参数属性，参数初始化，运算过程）
        self.layer = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
        
    def forward(self, x):
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = torch.sigmoid(out)
        return y_


import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

# Module对象
m = PerceptronModule()
# Module对象调用
y_ = m(x)

# 使用Module计算损失函数
loss = torch.nn.functional.mse_loss(y_, y)

# 优化损失-求导
loss.backward()

print(m.parameters())
print(list(m.parameters()))   # 子模块化方式：参数定义，属性设置，初始化都已经没子模块封装实现

<generator object Module.parameters at 0x135637990>
[Parameter containing:
tensor([[-0.4140, -0.1299, -0.0851, -0.1636]], requires_grad=True), Parameter containing:
tensor([0.1850], requires_grad=True)]


## 函数子模块

- 其中没有参数的函数采用不同的封装方式Function：
    - 主要管理求导
    
- 下面是例子代码

In [24]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 子模块对象
        # （简单的定义隐藏了参数定义，参数属性，参数初始化，运算过程）
        self.layer = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
        # ------------------
        self.sigmoid = torch.nn.Sigmoid()   # 函数对象
        # ------------------
        
    def forward(self, x):
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        # ---------------------
        y_ = self.sigmoid(out)
        # ---------------------
        return y_


import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

# Module对象
m = PerceptronModule()
# Module对象调用
y_ = m(x)

# 使用Module计算损失函数 ------------------------
l = torch.nn.MSELoss()     # 函数对象
loss = l(y_, y) 
# --------------------------------------------

# 优化损失-求导
loss.backward()
print(loss)

tensor(0.4384, grad_fn=<MseLossBackward>)


## 训练参数管理与优化器

- 训练参数管理主要包含两个：
    1. 梯度更新；
    2. 上次梯度清零；
    
- 这两个管理被封装到优化器中
    - 当然优化器还有其他的优化处理：比如学习率的动态变化等（可以加速提升训练过程）
    
- 下面是优化器的核心功能使用例子代码

In [50]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 子模块对象
        # （简单的定义隐藏了参数定义，参数属性，参数初始化，运算过程）
        self.layer = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
        self.sigmoid = torch.nn.Sigmoid()   # 函数对象
        
    def forward(self, x):
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = self.sigmoid(out)
        return y_


import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

epoch = 200
learning_rate = 0.005

# Module对象
m = PerceptronModule()
l = torch.nn.MSELoss()     # 函数对象
# 优化器对象
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)    # 构造器中需要模块的参数
# Module对象调用
for e in range(epoch):
    # 1. 梯度清零 --------------------------
    optimizer.zero_grad()
    
    y_ = m(x)
    loss = l(y_, y) 
    # 优化损失-求导
    loss.backward()
    
    # 2. 梯度更新--------------------------
    optimizer.step()
    with torch.autograd.no_grad():
        if e % 20 ==0:
            print("损失值：", loss.detach().numpy(), end="")
            predict = m(x)
            predict[predict>0.5] =1
            predict[predict<=0.5] =0
            print("\t测试准确度：", (predict == y).float().mean().detach().numpy())
    

损失值： 0.4592949	测试准确度： 0.0
损失值： 0.38074505	测试准确度： 0.02
损失值： 0.31565797	测试准确度： 0.42
损失值： 0.257681	测试准确度： 0.44
损失值： 0.20784816	测试准确度： 0.88
损失值： 0.16737387	测试准确度： 0.99
损失值： 0.13562103	测试准确度： 0.99
损失值： 0.11111626	测试准确度： 1.0
损失值： 0.092255495	测试准确度： 1.0
损失值： 0.07765004	测试准确度： 1.0


# Layer，函数与Module模块

## 函数模块的继承结构

- 使用MSELoss函数模块作为例子：

- `MSELoss`
    - `_Loss`
        - `torch.nn.modules.module.Module`
            - `builtins.object`

## Layer模块的继承结构

- 使用Liner Layer模块作为例子

- `Linear`
    - `torch.nn.modules.module.Module`
    - `builtins.object`

## Module模块

- 从上面的继承结构看出，函数模块与Layer模块都是Module，采用树状结构管理。
    - 函数模块比较简单：不带训练参数
    - Layer模块带训练参数

- Module类提供了数据结构管理
    1. 子模块的管理；
    2. 子模块的运算与运算过程管理；
    3. 子模块的求导逆向管理；
    4. 子模块训练参数管理；
    5. 子模块非训练参数管理；
    6. GPU/CPU计算方式切换管理；

### 子模块管理

- 子模块管理管理主要是数据结构的基本操作，子模块不负责形成决策运算，这个是forward函数负责。
    1. 添加子模块：
        - `add_module(self, name, module)`
    2. 返回子模块：
        - `children(self)`
        - `modules(self)`
        - `named_children(self)`
        - `named_modules(self, memo=None, prefix='')`
    3. 对某些模块的模式设置(主要是某些子模块的参数)
        - `eval(self)`
        - `train(self, mode=True)`

- 下面是子模块的管理的例子
    - 使用self定义成员属性，创建的都会纳入子模块管理。

In [60]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 子模块对象
        # （简单的定义隐藏了参数定义，参数属性，参数初始化，运算过程）
        self.layer = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
        self.sigmoid = torch.nn.Sigmoid()   # 函数对象
        
    def forward(self, x):
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = self.sigmoid(out)
        return y_


# Module对象
m = PerceptronModule()

# children成员函数
print(type(m.children()) ,list(m.children()))   # 两个子模块：返回的格式是生成器类型Generator
# modules成员函数：返回迭代器
print("------------------")
print(type(m.modules()), list(m.modules()))   # 返回所有的模块，包括PerceptronModule本身。
# named_children成员函数
print("------------------")
print(list(m.named_children()))
# named_modules成员函数
print("------------------")
print(list(m.named_modules()))

<class 'generator'> [Linear(in_features=4, out_features=1, bias=True), Sigmoid()]
------------------
<class 'generator'> [PerceptronModule(
  (layer): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
), Linear(in_features=4, out_features=1, bias=True), Sigmoid()]
------------------
[('layer', Linear(in_features=4, out_features=1, bias=True)), ('sigmoid', Sigmoid())]
------------------
[('', PerceptronModule(
  (layer): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)), ('layer', Linear(in_features=4, out_features=1, bias=True)), ('sigmoid', Sigmoid())]


- 子模块的添加与应用
    - 可以通过`add_module`添加，在forward中调用；（尽管不推荐，但在某些时候还是可以解决某些问题）

In [62]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 

    def forward(self, x):   # 1. 先试用
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = self.sigmoid(out)
        return y_


# Module对象
m = PerceptronModule()
layer1 = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
layer2 = torch.nn.Sigmoid()   # 函数对象
m.add_module("layer", layer1)     # 后添加
m.add_module("sigmoid", layer2)

import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

# Module对象调用
y_ = m(x[0:2])
print(y_)

tensor([[0.9001],
        [0.8923]], grad_fn=<SigmoidBackward>)


### 子模块的运算与运算过程管理

- 子模块的运算过程逻辑通过重载forward函数实现。
- 调用通过模块的可调用运算符实现()
    - `__call__`
- Module提供了回调的钩子机制：
    - `forward`调用后被回调：
        - `register_forward_hook(self, hook)`
            - 钩子函数原型：`hook(module, input, output) -> None or modified output`
    - `forward`被调用前回调：
        - `register_forward_pre_hook(self, hook)`
            - `hook(module, input) -> None or modified input`

In [63]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 

    def forward(self, x):   # 1. 先试用
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = self.sigmoid(out)
        return y_


# Module对象
m = PerceptronModule()
layer1 = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
layer2 = torch.nn.Sigmoid()   # 函数对象
m.add_module("layer", layer1)     # 后添加
m.add_module("sigmoid", layer2)

# ===========注册钩子函数
def after_hook(module, input, output):
    print("after============")
    print(module)
    print(input)
    print(output)
    # 如果想对输出做进一步处理，可以在这儿处理（比如其中view转换）
    return None   #-> None or modified output
def before_hook(module, input):
    print("before============")
    print(module)
    print(input)
    # 如果想对输出做进一步处理，可以在这儿处理（比如其中view转换）
    return None   #-> None or modified output

m.register_forward_hook(after_hook)
m.register_forward_pre_hook(before_hook)
# ==========================

import sklearn.datasets
# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

# Module对象调用
y_ = m(x[0:2])
print(y_)

before============
PerceptronModule(
  (layer): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000]]),)
after============
PerceptronModule(
  (layer): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000]]),)
tensor([[0.3483],
        [0.3723]], grad_fn=<SigmoidBackward>)
tensor([[0.3483],
        [0.3723]], grad_fn=<SigmoidBackward>)


### 子模块的求导逆向管理

- 提供了一个钩子函数，用来处理求导过程中的导数传递：
    - `register_backward_hook(self, hook)`
         - 钩子函数原型：`hook(module, grad_input, grad_output) -> Tensor or None`
             - 不需要直接修改这两个参数，而是返回新的梯度，返回的新的梯度会用作下一次的梯度输入。

- `register_backward_hook`函数是对整个Module的求导回调
    - 文档说：如果需要中间的梯度，则还是使用Tensor的钩子函数来获取争对某个变量的导数（可以参考我们前面的文档）

1. 对非子模块的情况

In [69]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 

    def forward(self, x, y, z):   # 1. 先试用
        xy = x * y
        xyz = torch.dot(xy, z)
        return xyz


# Module对象
m = PerceptronModule()
# ==========注册钩子函数
def back_hook(module, grad_input, grad_output):  
    print("输入梯度：", grad_input)
    print("输出梯度：", grad_output)
    return None
m.register_backward_hook(back_hook)
# ====================

x = torch.Tensor([1, 2, 3]) 
y = torch.Tensor([3, 4, 5]) 
z = torch.Tensor([4, 5, 6])
x.requires_grad=True
y.requires_grad=True
z.requires_grad=True


y_ = m(x, y, z)
y_.backward()
print(x.grad)
print(y.grad)
print(z.grad)

# 输出的是最后点积的导数。

输入梯度： (tensor([4., 5., 6.]), tensor([ 3.,  8., 15.]))
输出梯度： (tensor(1.),)
tensor([12., 20., 30.])
tensor([ 4., 10., 18.])
tensor([ 3.,  8., 15.])


2. 子模块的情况

In [79]:
import torch

class  PerceptronModule(torch.nn.Module):
    def __init__(self):
        super(PerceptronModule, self).__init__() 
        # 1. 子模块对象
        # （简单的定义隐藏了参数定义，参数属性，参数初始化，运算过程）
        self.layer = torch.nn.Linear(4, 1)    # 4 输入特征，3 输出特征
        self.sigmoid = torch.nn.Sigmoid()   # 函数对象
        
    def forward(self, x):
        # 2. 子模块计算过程的实现
        out = self.layer(x)   # 可调用对象
        y_ = self.sigmoid(out)
        return y_


import sklearn.datasets

# 加载鸢尾花数据集
data, target = sklearn.datasets.load_iris(return_X_y=True)
x = torch.Tensor(data[0:100])
y = torch.Tensor(target[0:100]).view(100, 1)

epoch = 200
learning_rate = 0.005

# Module对象
m = PerceptronModule()
# ==========注册钩子函数
def back_hook(module, grad_input, grad_output):  
    print("输入梯度：", type(grad_input), len(grad_input))
    print("输出梯度：", type(grad_output), len(grad_output))
    return None
m.register_backward_hook(back_hook)
# ====================
l = torch.nn.MSELoss()     # 函数对象
y_ = m(x)
loss = l(y_, y) 
loss.backward()

输入梯度： <class 'tuple'> 1
输出梯度： <class 'tuple'> 1


### 子模块训练参数管理

#### 训练参数的获取

- 训练参数在使用子模块是自动管理的，封装的优化器需要使用的参数，就通过下面的函数获取，可以使用如下函数获取：

```python

    parameters(self, recurse=True)
```


#### 参数的初始化

- 默认情况下训练参数的初始化都是模型的，但是用户需要初始化，可以指定函数来完成，函数的指定使用如下函数：

```python

    apply(self, fn)  
```

- 参数说明：
    - fn的函数原型是：`fn (Module) -> None`
        - 参数是Module，不需要返回值；

#### Conv2D模块中参数的默认初始化

```python
    def __init__(self, in_channels, out_channels, kernel_size, stride,
                 padding, dilation, transposed, output_padding,
                 groups, bias, padding_mode):
        super(_ConvNd, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        self.padding_mode = padding_mode
        if transposed:
            self.weight = Parameter(torch.Tensor(
                in_channels, out_channels // groups, *kernel_size))
        else:
            self.weight = Parameter(torch.Tensor(
                out_channels, in_channels // groups, *kernel_size))
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
# ---------------------
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
# ---------------------
```

#### Linear模块中参数的默认初始化

```python


    def __init__(self, in_features, out_features, bias=True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

```

#### BatchNorm2d模块的默认初始化

```python
    def reset_parameters(self):
        self.reset_running_stats()
        if self.affine:
            init.ones_(self.weight)
            init.zeros_(self.bias)
```

#### `init.kaiming_uniform_`函数

```python
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    fan = _calculate_correct_fan(tensor, mode)
    gain = calculate_gain(nonlinearity, a)
    std = gain / math.sqrt(fan)
    bound = math.sqrt(3.0) * std  # Calculate uniform bounds from standard deviation
    with torch.no_grad():
        return tensor.uniform_(-bound, bound)
```

```python
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out
```

- 可以参考我们前言中初始化，就是这个源代码的简化版本

#### 训练参数的其他函数

1. `zero_grad(self)`
2. `requires_grad_(self, requires_grad=True)`

### 子模块非训练参数管理

- 不参与训练的参数
    - requires_grad=False的Tesnor参数；
- 这个在Module中称为buffer

```python
    buffers(self, recurse=True)
```

```python
    named_buffers(self, prefix='', recurse=True)
```

### Module中其他函数  

#### GPU运算切换函数

1. `cuda(self, device=None)` 
2. `cpu(self)`
3. `to(self, *args, **kwargs)`


#### 类型转换函数

1. `to(self, *args, **kwargs)`
2. `half(self)`
3. `float(self)`
4. `double(self)`

# Layer与函数模块

- 这两个模块本质都是Module，但是Layer有训练参数，所以会提供一个函数完成参数初始化：
     - `reset_parameters(self)`
         - 可以重载该函数实现参数的定制初始化，淡然可以通过apply函数指定初始化函数也行。
     
     - 同时，他们的构造器因为作用的缘故，也存在一些差异。
     


## Torch提供的Layer与函数清单

- 这里只列出清单，因为使用与前面的函数一样，这里不累述。

1. Convolution layers
2. Pooling layers
3. Padding layers
4. Normalization layers
5. Recurrent layers
6. Transformer layers
7. Linear layers
8. Dropout layers
9. Sparse layers
10. Vision layers
11. Non-linear activations (weighted sum, nonlinearity)
12. Non-linear activations (other)
13. Distance functions
14. Loss functions

# 优化器模块

- 前面已经说明优化器的使用模式非常简单

## 参数设置

- 通过构造器设置

```python
    CLASS torch.optim.Optimizer(params, defaults)
```

- 参数说明：
    - params：来自Module的parameters函数返回的迭代器
    - defaults：使用字典管理的参数：
        - 典型的是学习率
        

## 参数管理的函数

1. 添加参数组：`add_param_group(param_group)`
2. 加载状态字典：`load_state_dict(state_dict)`
3. 返回状态字典：`state_dict()`

## 优化器的核心函数

1. 更新梯度：`step(closure)`
2. 清空梯度：`zero_grad()`

## 常见优化器

1. `torch.optim.Adadelta(params, lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)`
2. `torch.optim.Adagrad(params, lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)`
3. `torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)`
4. `torch.optim.AdamW(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)`
5. `torch.optim.SparseAdam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)`
6. `torch.optim.Adamax(params, lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)`
7. `torch.optim.ASGD(params, lr=0.01, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)`
8. `torch.optim.LBFGS(params, lr=1, max_iter=20, max_eval=None, tolerance_grad=1e-05, tolerance_change=1e-09, history_size=100, line_search_fn=None)`
9. `torch.optim.RMSprop(params, lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)`
10. `torch.optim.Rprop(params, lr=0.01, etas=(0.5, 1.2), step_sizes=(1e-06, 50))`
11. `torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)`


# 一个特殊用途的Module

- 在Torch提供了Sequential来组合多个Layer与函数。
    - 前面的输出需要匹配后面的输出。下面是一个简单的例子。

```python

    model = nn.Sequential(
        nn.Conv2d(1,20,5),
        nn.ReLU(),
        nn.Conv2d(20,64,5),、
        nn.ReLU()
    )

```

# 说明

1. 在torch中还提供了学习率修正的实现，这里暂时进一步说明，在后面高级主题中说明。
2. 还与Module有关的就是参数初始化函数的实现，这里也不详细介绍，相对比较简单
    - 请参考`torch.nn.init`模块中函数。

---- 